In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
from molsysmt import puw

# Adding solvent

In [3]:
pdbfile = msm.demo_systems.files['metenkephalin.pdb']

In [4]:
molecular_system = msm.convert(pdbfile)
molecular_system = msm.add_terminal_capping(molecular_system)
molecular_system = msm.remove_hydrogens(molecular_system)

In [5]:
msm.is_solvated(molecular_system)

False

In [6]:
molecular_system_solvated = msm.solvate([molecular_system, {'forcefield':'AMBER14',
                                                            'water_model':'TIP3P'}],
                                        box_geometry='cubic', clearance=14.0*puw.unit('angstroms'),
                                        verbose=False)

In [7]:
msm.is_solvated(molecular_system_solvated)

True

In [8]:
msm.info(molecular_system_solvated)

{'elements_item': <molsysmt.native.molsys.MolSys object at 0x7fc25d28e410>, 'elements_form': 'molsysmt.MolSys', 'bonds_item': <molsysmt.native.molsys.MolSys object at 0x7fc25d28e410>, 'bonds_form': 'molsysmt.MolSys', 'coordinates_item': <molsysmt.native.molsys.MolSys object at 0x7fc25d28e410>, 'coordinates_form': 'molsysmt.MolSys', 'velocities_item': None, 'velocities_form': None, 'box_item': <molsysmt.native.molsys.MolSys object at 0x7fc25d28e410>, 'box_form': 'molsysmt.MolSys', 'ff_parameters_item': None, 'ff_parameters_form': None, 'mm_parameters_item': None, 'mm_parameters_form': None, 'thermo_state_item': None, 'thermo_state_form': None, 'simulation_item': None, 'simulation_form': None}


form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_frames
molsysmt.MolSys,11646,3862,3858,1,3858,2,3857,1,1


In [9]:
n_waters, n_ions = msm.get(molecular_system_solvated, target='system',
                           n_waters=True, n_ions=True)
print('The molecular system has {} waters and {} ions.'.format(n_waters, n_ions))

The molecular system has 3857 waters and 0 ions.


With Openmm:

In [11]:
molecular_system_solvated = msm.solvate([msm.add_missing_hydrogens(molecular_system),
                                         {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                                        box_geometry='cubic', clearance=14.0*puw.unit('angstroms'),
                                        engine='OpenMM', verbose=False)

In [12]:
msm.info(molecular_system_solvated)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_frames
molsysmt.MolSys,9138,3026,3022,2,3022,2,3021,1,1


In [14]:
molecular_system_solvated = msm.solvate([molecular_system, {'forcefield':'AMBER14',
                                                           'water_model':'TIP3P'}],
                                        box_geometry='cubic', clearance=14.0*puw.unit('angstroms'),
                                        engine='PDBFixer', verbose=False)

In [15]:
msm.info(molecular_system_solvated)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_frames
molsysmt.MolSys,7891,2622,2618,2,2618,2,2617,1,1


Tengo que checar esto porque el número de aguas es muuuy distinto

In [16]:
molecular_system_solvated = msm.solvate([msm.add_missing_hydrogens(molecular_system),
                                         {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                                        box_geometry='truncated_octahedral',
                                        clearance=14.0*puw.unit('angstroms'),
                                        engine='OpenMM', verbose=False)

In [17]:
msm.info(molecular_system_solvated)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_frames
molsysmt.MolSys,2181,707,703,2,703,2,702,1,1


In [19]:
molecular_system_solvated = msm.solvate([msm.add_missing_hydrogens(molecular_system),
                                         {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                                        box_geometry='rhombic_dodecahedron',
                                        clearance=14.0*puw.unit('angstroms'),
                                        engine='OpenMM', verbose=False)

In [20]:
msm.info(molecular_system_solvated)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_frames
molsysmt.MolSys,1911,617,613,2,613,2,612,1,1
